## ECDS emergency care admission vs SUS hospital admission validation

There is a delay between patients being admitted to hospital with COVID-19 and this data being made available.  SUS-APCS data is the 'gold standard' for COVID-19 hospital admission but is only created once the patient is discharged (home, elswehere or died).  Data for ongoing hospital spells at the time of the SUS extract is therefore not available, creating an ascertainment bias against longer spells and more recent spells. 

This means we cannot rapidly evaluate vaccine effectiveness with respect to reducing hospital admission.

As a large proportion of hospital admission comes through A&E attendance, emergency admission data through ECDS may provide a good enough picture of hospital admission.  Below is a validation of this.

### Methods

* Patients hospitalised due to COVID-19 are first identified using SUS.
* Patients attending emergency care due to COVID-19 are identified using the criteria below:
    * All patients attending A&E AND then discharged to hospital or ICU are selected.
    * In these patients, those attending due to COVID-19 are filtered to include those that
        * Attend with positive COVID-19 code OR
        * Have had positive COVID-19 test in the 2 weeks prior to attendance OR
        * Have been recorded as COVID-19 positive in primary care 2 weeks prior to attendance (primary care positive test, primary care covid code, primary care covid sequalae). 
* Ability of ECDS data to identify those admitted to hospital due to COVID-19 vs SUS data is assessed.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib.colors import ListedColormap


%matplotlib inline

pd.options.display.float_format = '{:.0f}'.format

In [2]:
df = pd.read_csv('../output/input.csv')

#### Total SUS hospital admissions
Sus hospital admissions (any cause) that are identified by looking at ae attendance (any cause) that results in discharge to ward or icu

In [3]:
positive_patients_sus = df[df['hospital_admission'].notna()]
negative_patients_sus = df[~df['hospital_admission'].notna()]

positive_patients_ecds = df[(df['ae_attendance']==1)]
negative_patients_ecds = df[(df['ae_attendance']==0)]

total_sus_patients_positive = set(list(positive_patients_sus['patient_id']))
total_sus_patients_negative = set(list(negative_patients_sus['patient_id']))


ecds_patients_positive = set(list(positive_patients_ecds['patient_id']))
ecds_patients_negative = set(list(negative_patients_ecds['patient_id']))

sus_total_pos_ecds_pos = len(list(set(total_sus_patients_positive) & set(ecds_patients_positive)))
sus_total_pos_ecds_neg = len(list(set(total_sus_patients_positive) & set(ecds_patients_negative)))
sus_total_neg_ecds_pos = len(list(set(total_sus_patients_negative) & set(ecds_patients_positive)))
sus_total_neg_ecds_neg = len(list(set(total_sus_patients_negative) & set(ecds_patients_negative)))

In [4]:
print(f'+ hospitalisations SUS: {len(total_sus_patients_positive)}')
print(f'-  hospitalisations SUS: {len(total_sus_patients_negative)}')

+ hospitalisations SUS: 3000
-  hospitalisations SUS: 7000


In [5]:
pd.DataFrame([[sus_total_pos_ecds_pos, sus_total_neg_ecds_pos, (sus_total_pos_ecds_pos + sus_total_neg_ecds_pos)], [sus_total_pos_ecds_neg, sus_total_neg_ecds_neg, (sus_total_pos_ecds_neg + sus_total_neg_ecds_neg)], [(sus_total_pos_ecds_pos+sus_total_pos_ecds_neg), (sus_total_neg_ecds_pos+sus_total_neg_ecds_neg), (sus_total_pos_ecds_pos + sus_total_pos_ecds_neg + sus_total_neg_ecds_pos + sus_total_neg_ecds_neg)]], columns=["SUS-total-positive", "SUS-total-negative", "Total"], index=["ECDS-positive", "ECDS-negative", "Total"])


,SUS-total-positive,SUS-total-negative,Total
ECDS-positive,1175,2825,4000
ECDS-negative,1825,4175,6000
Total,3000,7000,10000


In [6]:
#sensitivity - number of sus identified by ecds
#specificity - number of those not in sus who are not in ecds

sensitivity = (sus_total_pos_ecds_pos/(sus_total_pos_ecds_pos + sus_total_pos_ecds_neg))*100
print(f"Sensitivity: {sensitivity:.2f}%")

specificity = (sus_total_neg_ecds_neg/(sus_total_neg_ecds_pos + sus_total_neg_ecds_neg))*100
print(f"Specificity : {specificity:.2f}%")

Sensitivity: 39.17%
Specificity : 59.64%


#### Covid SUS hospital admissions

In [7]:
positive_covid_patients_sus = df[df['primary_covid_hospital_admission'].notna()]
negative_covid_patients_sus = df[~df['primary_covid_hospital_admission'].notna()]

positive_covid_patients_ecds = df[(df['ae_attendance']==1) & ((df['ae_attendance_covid_status']==1) | (df['positive_covid_test_before_ae_attendance'] ==1) | (df['covid_primary_care_before_ae_attendance'] ==1))]
negative_covid_patients_ecds = df[(df['ae_attendance']==0) | ((df['ae_attendance']==1) & ((df['ae_attendance_covid_status']==0) & (df['positive_covid_test_before_ae_attendance'] ==0) & (df['covid_primary_care_before_ae_attendance'] ==0)))]


sus_patients_positive = set(list(positive_covid_patients_sus['patient_id']))
ecds_patients_positive = set(list(positive_covid_patients_ecds['patient_id']))

sus_patients_negative = set(list(negative_covid_patients_sus['patient_id']))
ecds_patients_negative = set(list(negative_covid_patients_ecds['patient_id']))


sus_pos_ecds_pos = len(list(set(sus_patients_positive) & set(ecds_patients_positive)))
sus_pos_ecds_neg = len(list(set(sus_patients_positive) & set(ecds_patients_negative)))
sus_neg_ecds_pos = len(list(set(sus_patients_negative) & set(ecds_patients_positive)))
sus_neg_ecds_neg = len(list(set(sus_patients_negative) & set(ecds_patients_negative)))

In [8]:
print(f'+ covid hospitalisations SUS: {len(sus_patients_positive)}')
print(f'- covid hospitalisations SUS: {len(sus_patients_negative)}')

+ covid hospitalisations SUS: 3000
- covid hospitalisations SUS: 7000


In [9]:
pd.DataFrame([[sus_pos_ecds_pos, sus_neg_ecds_pos, (sus_pos_ecds_pos + sus_neg_ecds_pos)], [sus_pos_ecds_neg, sus_neg_ecds_neg, (sus_pos_ecds_neg + sus_neg_ecds_neg)], [(sus_pos_ecds_pos+sus_pos_ecds_neg), (sus_neg_ecds_pos+sus_neg_ecds_neg), (sus_pos_ecds_pos + sus_pos_ecds_neg + sus_neg_ecds_pos + sus_neg_ecds_neg)]], columns=["SUS-positive", "SUS-negative", "Total"], index=["ECDS-positive", "ECDS-negative", "Total"])


,SUS-positive,SUS-negative,Total
ECDS-positive,1184,2713,3897
ECDS-negative,1816,4287,6103
Total,3000,7000,10000


In [10]:
#sensitivity - number of sus identified by ecds
#specificity - number of those not in sus who are not in ecds

sensitivity = (sus_pos_ecds_pos/(sus_pos_ecds_pos + sus_pos_ecds_neg))*100
print(f"Sensitivity: {sensitivity:.2f}%")

specificity = (sus_neg_ecds_neg/(sus_neg_ecds_pos + sus_neg_ecds_neg))*100
print(f"Specificity : {specificity:.2f}%")


Sensitivity: 39.47%
Specificity : 61.24%


## Variable Breakdown 

### AE Attendance (and subsequent discharge to ward or icu)


In [11]:
positive_ae_covid_ecds = df[((df['ae_attendance']==1))]
negative_ae_covid_ecds = df[((df['ae_attendance']==0))]


ecds_patients_positive = set(list(positive_ae_covid_ecds['patient_id']))
ecds_patients_negative = set(list(negative_ae_covid_ecds['patient_id']))

sus_pos_ecds_pos = len(list(set(sus_patients_positive) & set(ecds_patients_positive)))
sus_pos_ecds_neg = len(list(set(sus_patients_positive) & set(ecds_patients_negative)))
sus_neg_ecds_pos = len(list(set(sus_patients_negative) & set(ecds_patients_positive)))
sus_neg_ecds_neg = len(list(set(sus_patients_negative) & set(ecds_patients_negative)))

pd.DataFrame([[sus_pos_ecds_pos, sus_neg_ecds_pos, (sus_pos_ecds_pos + sus_neg_ecds_pos)], [sus_pos_ecds_neg, sus_neg_ecds_neg, (sus_pos_ecds_neg + sus_neg_ecds_neg)], [(sus_pos_ecds_pos+sus_pos_ecds_neg), (sus_neg_ecds_pos+sus_neg_ecds_neg), (sus_pos_ecds_pos + sus_pos_ecds_neg + sus_neg_ecds_pos + sus_neg_ecds_neg)]], columns=["SUS-positive", "SUS-negative", "Total"], index=["AE attendance +", "AE attendance -", "Total"])


,SUS-positive,SUS-negative,Total
AE attendance +,1224,2776,4000
AE attendance -,1776,4224,6000
Total,3000,7000,10000


In [12]:
sensitivity = (sus_pos_ecds_pos/(sus_pos_ecds_pos + sus_pos_ecds_neg))*100
print(f"Sensitivity: {sensitivity:.2f}%")

specificity = (sus_neg_ecds_neg/(sus_neg_ecds_pos + sus_neg_ecds_neg))*100
print(f"Specificity : {specificity:.2f}%")


Sensitivity: 40.80%
Specificity : 60.34%


### AE Attendance + AE Covid Status

In [13]:
positive_ae_covid_ecds = df[(df['ae_attendance']==1) & ((df['ae_attendance_covid_status']==1))]
negative_ae_covid_ecds = df[((df['ae_attendance']==0) | ((df['ae_attendance']==1) & (df['ae_attendance_covid_status']==0)))]



ecds_patients_positive = set(list(positive_ae_covid_ecds['patient_id']))
ecds_patients_negative = set(list(negative_ae_covid_ecds['patient_id']))

sus_pos_ecds_pos = len(list(set(sus_patients_positive) & set(ecds_patients_positive)))
sus_pos_ecds_neg = len(list(set(sus_patients_positive) & set(ecds_patients_negative)))
sus_neg_ecds_pos = len(list(set(sus_patients_negative) & set(ecds_patients_positive)))
sus_neg_ecds_neg = len(list(set(sus_patients_negative) & set(ecds_patients_negative)))

pd.DataFrame([[sus_pos_ecds_pos, sus_neg_ecds_pos, (sus_pos_ecds_pos + sus_neg_ecds_pos)], [sus_pos_ecds_neg, sus_neg_ecds_neg, (sus_pos_ecds_neg + sus_neg_ecds_neg)], [(sus_pos_ecds_pos+sus_pos_ecds_neg), (sus_neg_ecds_pos+sus_neg_ecds_neg), (sus_pos_ecds_pos + sus_pos_ecds_neg + sus_neg_ecds_pos + sus_neg_ecds_neg)]], columns=["SUS-positive", "SUS-negative", "Total"], index=["AE Covid +", "AE Covid -", "Total"])



,SUS-positive,SUS-negative,Total
AE Covid +,1101,2502,3603
AE Covid -,1899,4498,6397
Total,3000,7000,10000


In [14]:
sensitivity = (sus_pos_ecds_pos/(sus_pos_ecds_pos + sus_pos_ecds_neg))*100
print(f"Sensitivity: {sensitivity:.2f}%")

specificity = (sus_neg_ecds_neg/(sus_neg_ecds_pos + sus_neg_ecds_neg))*100
print(f"Specificity : {specificity:.2f}%")


Sensitivity: 36.70%
Specificity : 64.26%


###  AE Attendance + Recent Positive Covid Test

In [15]:
positive_cov_test_ecds = df[(df['ae_attendance']==1) & ((df['positive_covid_test_before_ae_attendance']==1))]
negative_cov_test_ecds = df[((df['ae_attendance']==0) | ((df['ae_attendance']==1) & (df['positive_covid_test_before_ae_attendance']==0)))]


ecds_patients_positive = set(list(positive_cov_test_ecds['patient_id']))
ecds_patients_negative = set(list(negative_cov_test_ecds['patient_id']))

sus_pos_ecds_pos = len(list(set(sus_patients_positive) & set(ecds_patients_positive)))
sus_pos_ecds_neg = len(list(set(sus_patients_positive) & set(ecds_patients_negative)))
sus_neg_ecds_pos = len(list(set(sus_patients_negative) & set(ecds_patients_positive)))
sus_neg_ecds_neg = len(list(set(sus_patients_negative) & set(ecds_patients_negative)))

pd.DataFrame([[sus_pos_ecds_pos, sus_neg_ecds_pos, (sus_pos_ecds_pos + sus_neg_ecds_pos)], [sus_pos_ecds_neg, sus_neg_ecds_neg, (sus_pos_ecds_neg + sus_neg_ecds_neg)], [(sus_pos_ecds_pos+sus_pos_ecds_neg), (sus_neg_ecds_pos+sus_neg_ecds_neg), (sus_pos_ecds_pos + sus_pos_ecds_neg + sus_neg_ecds_pos + sus_neg_ecds_neg)]], columns=["SUS-positive", "SUS-negative", "Total"], index=["Covid Test +", "Covid Test -", "Total"])


,SUS-positive,SUS-negative,Total
Covid Test +,604,1410,2014
Covid Test -,2396,5590,7986
Total,3000,7000,10000


In [16]:
sensitivity = (sus_pos_ecds_pos/(sus_pos_ecds_pos + sus_pos_ecds_neg))*100
print(f"Sensitivity: {sensitivity:.2f}%")

specificity = (sus_neg_ecds_neg/(sus_neg_ecds_pos + sus_neg_ecds_neg))*100
print(f"Specificity : {specificity:.2f}%")


Sensitivity: 20.13%
Specificity : 79.86%


### AE Attendance + Covid Positive Primary Care

In [17]:
positive_cov_pc_ecds = df[(df['ae_attendance']==1) & ((df['covid_primary_care_before_ae_attendance']==1))]
negative_cov_pc_ecds = df[((df['ae_attendance']==0) | ((df['ae_attendance']==1) & (df['covid_primary_care_before_ae_attendance']==0)))]


ecds_patients_positive = set(list(positive_cov_pc_ecds['patient_id']))
ecds_patients_negative = set(list(negative_cov_pc_ecds['patient_id']))

sus_pos_ecds_pos = len(list(set(sus_patients_positive) & set(ecds_patients_positive)))
sus_pos_ecds_neg = len(list(set(sus_patients_positive) & set(ecds_patients_negative)))
sus_neg_ecds_pos = len(list(set(sus_patients_negative) & set(ecds_patients_positive)))
sus_neg_ecds_neg = len(list(set(sus_patients_negative) & set(ecds_patients_negative)))

pd.DataFrame([[sus_pos_ecds_pos, sus_neg_ecds_pos, (sus_pos_ecds_pos + sus_neg_ecds_pos)], [sus_pos_ecds_neg, sus_neg_ecds_neg, (sus_pos_ecds_neg + sus_neg_ecds_neg)], [(sus_pos_ecds_pos+sus_pos_ecds_neg), (sus_neg_ecds_pos+sus_neg_ecds_neg), (sus_pos_ecds_pos + sus_pos_ecds_neg + sus_neg_ecds_pos + sus_neg_ecds_neg)]], columns=["SUS-positive", "SUS-negative", "Total"], index=["Primary Care Covid +", "Primary Care Covid -", "Total"])


,SUS-positive,SUS-negative,Total
Primary Care Covid +,618,1352,1970
Primary Care Covid -,2382,5648,8030
Total,3000,7000,10000


In [18]:
sensitivity = (sus_pos_ecds_pos/(sus_pos_ecds_pos + sus_pos_ecds_neg))*100
print(f"Sensitivity: {sensitivity:.2f}%")

specificity = (sus_neg_ecds_neg/(sus_neg_ecds_pos + sus_neg_ecds_neg))*100
print(f"Specificity : {specificity:.2f}%")


Sensitivity: 20.60%
Specificity : 80.69%
